# WNTR Earthquake Tutorial
The following tutorial covers how to run simple earthquake analysis with partial repairs using WNTR.

## Imports
Import WNTR and additional Python packages that are needed for the tutorial.
- Numpy is required to define comparison operators (i.e., np.greater) in queries
- Pandas is used for data manipulation and analysis
- Matplotlib is used to create graphics
- Scipy.stats is used for a wide range of statistical computations

In [ ]:
import wntr
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from scipy.stats import expon


## Water network model
If adapting the code for a different EPANET input (INP) file, the correct file path and desired simulation parameters will need to be updated.

In [ ]:
# Identify file path to inp file
inp_file = "networks/Net3.inp"

# Create water network model 
wn = wntr.network.WaterNetworkModel(inp_file)

# Morph network coordinates, this is optional
wn = wntr.morph.scale_node_coordinates(wn, 1000) 

## Earthquake scenario
The earthquake scenario is simulated by defining the characteristics of the earthquake, including the epicenter, magnitude and depth. After defining the earthquake scenario, the earthquake is simulated to calculate the peak ground acceleration (PGA), peak ground velocity (PGV), and repair rates (RR). Fragility curves are then used to define the probability of damage with respect to PGA, PGV, and RR.

In [ ]:
# Define earthquake simulation parameters 
epicenter = (32000,15000) # m (x,y)
magnitude = 6.5 # Richter magnitude
depth = 10000 # m

total_duration = 24*3600 # 24 hours

minimum_pressure = 3.52 # 5 psi
required_pressure = 14.06 # 20 psi

leak_start_time = 5*3600 # 5 hours
leak_repair_time =  15*3600 # 15 hours

np.random.seed(12345)

In [ ]:
# Plot the epicenter on water network model
fig, ax = plt.subplots()
ax.scatter(epicenter[0], epicenter[1], s=500, color='r', marker='*', zorder=2)
wntr.graphics.plot_network(wn, node_size=0, ax=ax)

In [ ]:
# Initialize earthquake object
earthquake = wntr.scenario.Earthquake(epicenter, magnitude, depth) 

# Calculate peak ground acceleration (PGA), peak ground velocity (PGV), and repair rates (RR)
R = earthquake.distance_to_epicenter(wn, element_type=wntr.network.Pipe)
pga = earthquake.pga_attenuation_model(R)  
pgv = earthquake.pgv_attenuation_model(R)
RR = earthquake.repair_rate_model(pgv)

# Query pipe lengths for all pipes in the network
L = pd.Series(wn.query_link_attribute('length', link_type = wntr.network.Pipe))

# Generate fragility curve
pipe_FC = wntr.scenario.FragilityCurve()
pipe_FC.add_state('Minor Leak', 1, {'Default': expon(scale=0.2)})
pipe_FC.add_state('Major Leak', 2, {'Default': expon()})

In [ ]:
# Plot the fragility curve
plt.figure()
wntr.graphics.plot_fragility_curve(pipe_FC, title = 'Fragility Curve', xlabel='Rate of repair * pipe length')

In [ ]:
# Calculate leak probabilities using RR and fragility curve  
pipe_Pr = pipe_FC.cdf_probability(RR*L)
pipe_damage_state = pipe_FC.sample_damage_state(pipe_Pr)

# Print the minimum, maximum, and average PGA, PGV, RR, and RR*pipelength
print("Min, Max, Average PGA: " + str(np.round(pga.min(),2)) + ", " + str(np.round(pga.max(),2)) + ", " + str(np.round(pga.mean(),2)) + " g")
print("Min, Max, Average PGV: " + str(np.round(pgv.min(),2)) + ", " + str(np.round(pgv.max(),2)) + ", " + str(np.round(pgv.mean(),2)) + " m/s")
print("Min, Max, Average repair rate: " + str(np.round(RR.min(),5)) + ", " + str(np.round(RR.max(),5)) + ", " + str(np.round(RR.mean(),5)) + " per m")
print("Min, Max, Average repair rate*pipe length: " + str(np.round((RR*L).min(),5)) + ", " + str(np.round((RR*L).max(),5)) + ", " + str(np.round((RR*L).mean(),5)))

### Plots
The results of the earthquake scenario are plotted on the water distribution network map. The results include the distance to the earthquake epicenter, the PGA, the PGV, the RR, the RR*pipe length, and the probabilities of minor and major leaks. 

In [ ]:
# Set colormap for network maps
cmap=matplotlib.colormaps['viridis']

# Plot distance to epicenter
wntr.graphics.plot_network(wn, link_attribute=R, node_size=0, link_cmap = cmap, link_colorbar_label='Distance (m)', title='Distance to Epicenter')

# Plot PGA
wntr.graphics.plot_network(wn, link_attribute=pga, node_size=0, link_cmap = cmap, link_width=1.5, link_colorbar_label='PGA (g)', title='Peak Ground Acceleration (PGA)')

# Plot PGV
wntr.graphics.plot_network(wn, link_attribute=pgv, node_size=0, link_cmap = cmap, link_width=1.5, link_colorbar_label='PVA (m/s)', title='Peak Ground Velocity (PVA)')

# Plot RR (# of repairs needed per m)
wntr.graphics.plot_network(wn, link_attribute=RR, node_size=0, link_cmap = cmap, link_width=1.5, link_colorbar_label='RR (per m)', title='Repair Rate')

# Plot RR*Pipe Length
wntr.graphics.plot_network(wn, link_attribute=(RR*L), node_size=0, link_cmap = cmap, link_width=1.5, link_colorbar_label='RR*Length', title='Repair Rate*Pipe Length')

# Plot probability of minor leak
wntr.graphics.plot_network(wn, link_attribute=pipe_Pr['Minor Leak'], node_size=0, link_cmap = cmap, link_range=[0,1], link_width=1.5, link_colorbar_label='Probability', title='Probability of a Minor Leak')

# Plot probability of major leak
wntr.graphics.plot_network(wn, link_attribute=pipe_Pr['Major Leak'], node_size=0, link_cmap = cmap, link_range=[0,1], link_width=1.5, link_colorbar_label='Probability', title='Probability of a Major Leak')

## Earthquake hydraulic simulation
The hydraulic effects of the earthquake scenario are then simulated on the water distribution network model. Two conditions are simulated: without repairs to pipes damaged by the earthquake and with repairs to the damaged pipes with the largest leaks. 


### Without repairs

In [ ]:
# Set simulation parameters
wn.options.hydraulic.demand_model = 'PDD'    
wn.options.time.duration = total_duration
wn.options.hydraulic.minimum_pressure = minimum_pressure
wn.options.hydraulic.required_pressure = required_pressure 

# Define leaks according to the size 
for pipe_name, damage_state in pipe_damage_state.items():
    pipe_diameter = wn.get_link(pipe_name).diameter
    if damage_state is not None:
        if damage_state == 'Major Leak':
            leak_diameter = 0.25*pipe_diameter 
            leak_area = np.pi/4.0*leak_diameter**2
        elif damage_state == 'Minor Leak':
            leak_diameter = 0.1*pipe_diameter 
            leak_area = np.pi/4.0*leak_diameter**2
        else:
            leak_area = 0

        # Add pipe leak to network
        wn = wntr.morph.split_pipe(wn,pipe_name, pipe_name+'A', 'Leak'+pipe_name)
        n = wn.get_node('Leak'+pipe_name)
        n.add_leak(wn, area=leak_area, start_time=leak_start_time)

# Simulate the hydraulics   
sim = wntr.sim.WNTRSimulator(wn)
results = sim.run_sim()

### Largest leaks for repair

In [ ]:
# Rank leaked demand
leaked_demand = results.node['leak_demand']
leaked_sum = leaked_demand.sum()
leaked_sum.sort_values(ascending=False, inplace=True)

# Plot leak demand
leaked_demand.index = leaked_demand.index/3600
ax=leaked_demand.plot(legend=False)
ax.set_ylabel('Leak demand (m$^3$/s)')
ax.set_xlabel('Time (hr)')

In [ ]:
# Select top pipes to fix
number_of_pipes_to_repair = 4
pipes_to_fix = leaked_sum[0:number_of_pipes_to_repair]
print(pipes_to_fix)

### With repairs
When using the same water network model to run multiple simulations (with WNTRSimulator), initial conditions need to be reset between simulations. Initial conditions include simulation time, tank head, reservoir head, pipe status, pump status, and valve status.

In [ ]:
# Reset to initial conditions
wn.reset_initial_values()

# Identify largest leaks
leaked_demand = results.node['leak_demand']
leaked_sum = leaked_demand.sum()
leaked_sum.sort_values(ascending=False, inplace=True)

# Specify the number of leaks to repair
number_of_pipes_to_repair = 4
leaks_to_fix = leaked_sum[0:number_of_pipes_to_repair]

# Simulate a partial fixed leak
for leak_name in leaks_to_fix.index:
    node = wn.get_node(leak_name)
    leak_area = node.leak_area
    node.remove_leak(wn)
    node.add_leak(wn, area=leak_area, start_time=leak_start_time, end_time=leak_repair_time)

# Simulate partial repairs
results_wrepair = sim.run_sim()

## Results
The network pressure, the water service availablity (WSA), and the population impacted are calculated and plotted for the earthquake scenarios without repairs and with partial repairs simulated. 

### Network pressure

In [ ]:
pressure = results.node['pressure']
pressure_wrepair = results_wrepair.node['pressure']
pressure.index = pressure.index/3600
pressure_wrepair.index = pressure_wrepair.index/3600

# Plot pressure 24 hr into simulation without repair
pressure_at_24hr = pressure.loc[24,wn.junction_name_list]
wntr.graphics.plot_network(wn, node_attribute=pressure_at_24hr, node_size=20, 
                           node_range=[0,90], node_cmap = cmap, node_colorbar_label='Pressure (m)', 
                           title='Pressure at 24 hours, Without Repair')

# Plot pressure 24 hr into simulation with partial repair
pressure_at_24hr_wrepair = pressure_wrepair.loc[24,wn.junction_name_list]
wntr.graphics.plot_network(wn, node_attribute=pressure_at_24hr_wrepair, node_size=20, 
                           node_range=[0,90], node_cmap = cmap, node_colorbar_label='Pressure (m)', 
                           title='Pressure at 24 hours, With Repair')
 
# Plot average system pressure with and without partial repairs
plt.figure()
ax = plt.gca()
pressure.loc[:,wn.junction_name_list].mean(axis=1).plot(label='Without Repair', ax=ax)
pressure_wrepair.loc[:,wn.junction_name_list].mean(axis=1).plot(label='With Repair', ax=ax)
ax.set_xlabel('Time (hr)')
ax.set_ylabel('Average system pressure (m)')
ax.legend()

### Water service availability

In [ ]:
expected_demand = wntr.metrics.expected_demand(wn)
demand = results.node['demand'].loc[:,wn.junction_name_list]
demand_wrepair = results_wrepair.node['demand'].loc[:,wn.junction_name_list]
expected_demand.index = expected_demand.index/3600
demand.index = demand.index/3600
demand_wrepair.index = demand_wrepair.index/3600

# Calculate water service availability (WSA)
wsa = wntr.metrics.water_service_availability(expected_demand, demand)
wsa_wrepair = wntr.metrics.water_service_availability(expected_demand, demand_wrepair)

# Plot WSA
fig, axarr = plt.subplots(1,2,figsize=(14,4))
ax = axarr[0]
wsa.plot(ax=ax, legend=False)
ax.set_ylim(ymin=-0.05, ymax=1.05)
ax.set_xlabel('Time (hr)')
ax.set_ylabel('Water service availability')
ax.set_title('Without Repair')
ax = axarr[1]
wsa_wrepair.plot(ax=ax, legend=False)
ax.set_ylim(ymin=-0.05, ymax=1.05)
ax.set_xlabel('Time (hr)')
ax.set_ylabel('Water service availability')
ax.set_title('With Repair')

### Population impacted

In [ ]:
# Note that water service availability of NaN is replaced with 0 for the population impacted calculation
population = wntr.metrics.population(wn)
people_impacted = wntr.metrics.population_impacted(population, wsa.fillna(0), np.less, 0.8)
people_impacted_wrepair = wntr.metrics.population_impacted(population, wsa_wrepair.fillna(0), np.less, 0.8)

# Plot population impacted
fig, axarr = plt.subplots(1,2,figsize=(14,4))
ax = axarr[0]
people_impacted.plot(ax=ax, legend=False)
ax.set_ylim(ymax=35000)
ax.set_xlabel('Time (hr)')
ax.set_ylabel('Population impacted')
ax.set_title('Without Repair')
ax = axarr[1]
people_impacted_wrepair.plot(ax=ax, legend=False)
ax.set_ylim(ymax=35000)
ax.set_xlabel('Time (hr)')
ax.set_ylabel('Population impacted')
ax.set_title('With Repair')

## Save results to CSV files

In [ ]:
# Save the population impacted without repairs results to CSV
# Check to verify the file was created in the directory
people_impacted.to_csv('earthquake_people_impacted.csv')

# Save the population impacted with repairs results to CSV
people_impacted_wrepair.to_csv('earthquake_people_impacted_wrepair.csv')